### import the data


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_path = r"C:\Users\paul zhan\Downloads\vegetable dataset\Vegetable Images"

# Define the paths to your train, test, and validation folders
train_path = r"C:\Users\paul zhan\Desktop\vegetable model\vegetable dataset\Vegetable Images\train"
test_path = r"C:\Users\paul zhan\Desktop\vegetable model\vegetable dataset\Vegetable Images\test"
validation_path = r"C:\Users\paul zhan\Desktop\vegetable model\vegetable dataset\Vegetable Images\validation"

# Create ImageDataGenerators for each set and doing the normalisation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load the data using flow_from_directory method
# using flow from directory to access the sub folder
train_data = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_data = test_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_data = validation_datagen.flow_from_directory(validation_path, target_size=(224, 224), batch_size=32, class_mode='categorical')

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


### Now i am going to make the model


In [5]:
from tensorflow import keras
num_classes = 15

model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])


In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Retrieve the first batch from train_data
first_batch = next(train_data)

# Access the images and labels of the first batch
batch_images = first_batch[0]
batch_labels = first_batch[1]

# Check the shape of the batch
print(batch_images.shape)  # Shape of the input images in the batch
print(batch_labels.shape)  # Shape of the corresponding labels in the batch

(32, 224, 224, 3)
(32, 15)


### now I am going to train the model 

In [8]:
model.fit(train_data, epochs=5, validation_data=validation_data)

Epoch 1/5
469/469 [==============================] - 437s 929ms/step - loss: 1.1311 - accuracy: 0.6841 - val_loss: 0.5179 - val_accuracy: 0.8440
Epoch 2/5
469/469 [==============================] - 434s 925ms/step - loss: 0.2599 - accuracy: 0.9231 - val_loss: 0.2945 - val_accuracy: 0.9107
Epoch 3/5
469/469 [==============================] - 369s 786ms/step - loss: 0.0977 - accuracy: 0.9722 - val_loss: 0.3139 - val_accuracy: 0.9090
Epoch 4/5
469/469 [==============================] - 362s 771ms/step - loss: 0.0453 - accuracy: 0.9883 - val_loss: 0.3828 - val_accuracy: 0.8963
Epoch 5/5
469/469 [==============================] - 360s 768ms/step - loss: 0.0674 - accuracy: 0.9793 - val_loss: 0.3293 - val_accuracy: 0.9280


In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6743045440732017655
xla_global_id: -1
]


In [10]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_data)

# Print the test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

94/94 [==============================] - 16s 169ms/step - loss: 0.3455 - accuracy: 0.9220
Test Loss: 0.34545689821243286
Test Accuracy: 0.921999990940094


In [11]:
import cv2
# Open the camera
cap = cv2.VideoCapture(0)  # Replace 0 with the camera index if you have multiple cameras

## now using real time camera

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np

# Load the trained model
# model = keras.models.load_model('path_to_your_trained_model.h5')

# Define the labels for your classes
class_labels = ['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']  # Replace with your class labels

# Function to preprocess the image
def preprocess_image(img):
    img = cv2.resize(img, (224, 224))  # Resize the image to match the input shape of your model using OpenCV
    img = img / 255.0  # Normalize the pixel values
    img = np.expand_dims(img, axis=0)  # Add an extra dimension to represent the batch
    return img

# Function to predict the class of an image
def predict_class(img):
    img = preprocess_image(img)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    predicted_label = class_labels[predicted_class]
    return predicted_label

# Open the camera
cap = cv2.VideoCapture(0)  # Replace 0 with the camera index if you have multiple cameras

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Display the resulting frame
    cv2.imshow('Camera', frame)
    
    # Preprocess the frame and predict the class
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert the frame from BGR to RGB format
    predicted_label = predict_class(img)
    
    # Display the predicted class label
    cv2.putText(frame, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Camera', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 33ms/step
